<a href="https://colab.research.google.com/github/vnicula/ai-research-code/blob/master/algorithms/multivariate_regression_sgd_hidden_layer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


--2022-11-14 07:49:17--  https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/FuelConsumptionCo2.csv
Resolving s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)... 67.228.254.196
Connecting to s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 72629 (71K) [text/csv]
Saving to: ‘FuelConsumption.csv’

FuelConsumption.csv 100%[===================>]  70.93K  --.-KB/s    in 0.03s   

2022-11-14 07:49:17 (2.22 MB/s) - ‘FuelConsumption.csv’ saved [72629/72629]



In [6]:
df = pd.read_csv('https://raw.githubusercontent.com/satishgunjal/datasets/master/multivariate_housing_prices_in_portlans_oregon.csv')
print(df.size)
df.head()

141


,size(in square feet),number of bedrooms,price
0,2104,3,399900
1,1600,3,329900
2,2400,3,369000
3,1416,2,232000
4,3000,4,539900


In [15]:
X_train = df.iloc[:, 0:2].values
# print(np.std(X_train, axis=0, keepdims=True)[:3])
Y_train = np.expand_dims(df.iloc[:, 2].values, 1)
X_train = (X_train - X_train.mean(axis=0, keepdims=True)) / np.std(X_train, axis=0, keepdims=True)
print(X_train[:3], Y_train[:3])


[[7.86202619e+02 7.52842809e-01]]
[[ 0.13141542 -0.22609337]
 [-0.5096407  -0.22609337]
 [ 0.5079087  -0.22609337]] [[399900]
 [329900]
 [369000]]


In [16]:
def init_weights():
  W1 = np.random.normal(loc=0, scale=0.2, size=[2, 3])
  B1 = np.random.normal(loc=0, scale=0.2, size=[1, 3])
  W2 = np.random.normal(loc=0, scale=0.2, size=[3, 1])
  B2 = np.random.normal(loc=0, scale=0.2)
  return W1, B1, W2, B2

# print(init_weights())

(array([[ 0.37397902,  0.22600203,  0.0659727 ],
       [ 0.05244676,  0.16280207, -0.02475934]]), array([[ 0.32143545, -0.06737601, -0.25961598]]), array([[0.01614507],
       [0.06377955],
       [0.35477123]]), -0.1001778459210112)


In [21]:
def ReLU(z):
  return np.maximum(z, 0)

def loss(Y, Yh):
  return 0.5*(Y-Yh)**2

def forward_pass(X, W1, B1, W2, B2):
  Z1 = X.dot(W1) + B1
  A1 = ReLU(Z1)
  O = A1.dot(W2) + B2
  return Z1, A1, O


In [ ]:
def backward_pass(X, Y, Z1, A1, O, W1, B1, W2, B2):
  dL_O = Y - O # (m, 1)
  dO_W2 = A1   # (m, 3)
  dO_B2 = 1
  dO_A1 = W2   # (1, 3)
  dA1_Z1 = (Z1 > 0) * 1.0  # (m, 3)
  dZ1_W1 = X   # (m, 3, 2)

  gW2 = dL_O.T.dot(dO_W2)
  gB2 = dL_O.T.dot(np.ones_like(dL_O))

  gW1 = dL_O.dot(dO_A1).T.dot(dA1_Z1).dot(dZ1_W1)
  gB1 = dL_O.dot(dO_A1).T.dot(dA1_Z1)